In [ ]:
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import json

import plotly.express as px

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Load lsoa data
lsoas = gpd.read_file('../geodata/LSOAs.geojson').to_crs(epsg=4326)
lsoas = (
    lsoas[lsoas['LAD11NM'] != 'City of London'][['LSOA11CD','geometry']]
    .rename(columns={'LSOA11CD':'lsoa_code'})
)

# Load preprocessed crime data
data = pd.read_csv('../data/predictive_data.csv')

In [ ]:
# Per-LSOA temporal split
def lsoa_wise_temporal_split(df, test_months=3):
    train_list, test_list = [], []
    # for each lsoa and its subdf group
    for lsoa, group in df.groupby('lsoa_code'):
        group = group.sort_values('month')
        # if the lsoa has less than or equal to amount of test months always put in train list
        if group.shape[0] <= test_months:
            train_list.append(group)
        # else there is enough data, so add all but last test_months rows to training
        # add last test_months rows to test
        else:
            train_list.append(group.iloc[:-test_months])
            test_list.append(group.iloc[-test_months:])
    # convert into dfs
    train_df = pd.concat(train_list)
    test_df = pd.concat(test_list)
    return train_df, test_df

In [ ]:
# independent variables
features = [
    'rolling_std_3','rolling_mean_6','rolling_sum_12',
    'Anti-social behaviour', 'Bicycle theft', 'Criminal damage and arson',
    'Drugs', 'Other crime', 'Other theft',
    'Public order', 'Robbery', 'Shoplifting', 'Theft from the person',
    'Vehicle crime', 'Violence and sexual offences',
    'Possession of weapons', 'rolling_mean_3','health_decile_2019',
    'lag_1','lag_2','lag_3','imd_decile_2019','income_decile_2019','employment_decile_2019',
    'crime_decile_2019',
]

# get train and test dfs 
train_df, test_df = lsoa_wise_temporal_split(data, test_months=3)
print(f"Train rows: {len(train_df)}, Test rows: {len(test_df)}")

X_train = train_df[features]
y_train = train_df[['target_1','target_2','target_3']]
X_test  = test_df[features]
y_test  = test_df[['target_1','target_2','target_3']]

In [ ]:
# # Check for NAN values
# print("X_train missing per column:\n", X_train.isna().sum())
# print("X_test  missing per column:\n", X_test.isna().sum())

# print("y_train missing:\n", y_train.isna().sum())
# print("y_test  missing:\n", y_test.isna().sum())


In [ ]:
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(3,  activation='linear')
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

t0 = time.perf_counter()    # timer for fitting
history = model.fit(
    X_train, y_train.values,
    validation_split=0.1,
    epochs=100,
    batch_size=256,
    callbacks=[es],
    verbose=0
)
fit_time = time.perf_counter() - t0
print(f'Fit time: {fit_time:.2f} seconds')

y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
horizons = ['1‑month ahead', '2‑months ahead', '3‑months ahead']
for i, title in enumerate(horizons):
    rmse = np.sqrt(mean_squared_error(y_test.values[:,i], y_pred[:,i]))
    r2   = r2_score(y_test.values[:,i], y_pred[:,i])
    print(f'{title}: RMSE={rmse:.2f}, R²={r2:.3f}')

In [ ]:
months_ahead = 3

# Store predictions for visualization
vis_df = test_df.copy()
vis_df[['pred_1', 'pred_2', 'pred_3']] = y_pred

latest_preds = (
    vis_df.sort_values('month')
           .groupby('lsoa_code')
           .tail(1)[['lsoa_code', f'pred_{months_ahead}']]
           .rename(columns={f'pred_{months_ahead}': 'predicted_burglaries'})
)

latest_preds.reset_index(drop=True, inplace=True)

# Merge predictions with GeoData
geo = lsoas.merge(latest_preds, on='lsoa_code', how='inner')

# Convert to JSON
geojson = json.loads(geo.to_json())

# Plot choropleth (works in Jupyter)
fig = px.choropleth_map(
    geo,
    geojson=geojson,
    locations='lsoa_code',
    featureidkey="properties.lsoa_code",
    color='predicted_burglaries',
    range_color=(0, geo['predicted_burglaries'].max()),
    color_continuous_scale="OrRd",
    map_style="open-street-map",
    zoom=9,
    center={"lat": 51.5072, "lon": -0.1276},
    opacity=0.6,
    height=600
)

fig.update_layout(title=f'Predicted Burglary Count ({months_ahead}-month horizon) by LSOA')
fig.show()